In [1]:
import alpaca
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient

from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest    
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus 
)
from alpaca.data.historical.option import OptionHistoricalDataClient


from dotenv import load_dotenv
import os

# keys is stored in config.env file
# - Specify the path to the config.env file
dotenv_path = os.path.join(os.getcwd(), "config.env")
# Load environment variables from the specified .env file
load_dotenv(dotenv_path)

# API credentials for Alpaca
API_KEY = os.getenv("ALPACA_API_KEY_PAPER")
API_SECRET = os.getenv('ALPACA_API_SECRET_KEY_PAPER')
BASE_URL = None
PAPER = True

# Initialize Alpaca API
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)

# Get option chain by underlying_symbol
option_historical_data_client = OptionHistoricalDataClient(API_KEY, API_SECRET, url_override = BASE_URL)

In [17]:
# get put options contracts
# - in this example, we get put options contracts for SPY = SPDR S&P 500 ETF
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)

# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

KeyboardInterrupt: 

In [ ]:
symbol = 'SPY'
# Function to check for 0DTE options
def get_0DTE_options(symbol):
    # Fetch options data (hypothetically, as Alpaca doesn’t yet have options)
    req = OptionChainRequest(underlying_symbol = symbol)
    # Get option chain of the underlying symbol
    options_chain = option_historical_data_client.get_option_chain(req)
    # Extract today's date in the YYMMDD format
    today_str = datetime.now().strftime('%y%m%d') + 1
    # Filter options with 0 days to expiration
    zero_dte_options = []
    for option_symbol, option_data in options_chain.items():
        if today_str in option_symbol:
            zero_dte_options.append(option_data)
    return zero_dte_options

get_0DTE_options(symbol)

APIError: {"message": "endpoint not found."}


In [ ]:
# Function to check for 0DTE options
def get_0DTE_options(symbol):
    # Fetch options data (hypothetically, as Alpaca doesn’t yet have options)
    req = OptionChainRequest(underlying_symbol = symbol)
    # Get option chain of the underlying symbol
    options_chain = option_historical_data_client.get_option_chain(req)
    # Extract today's date in the YYMMDD format
    today_str = datetime.now().strftime('%y%m%d')
    # Filter options with 0 days to expiration
    zero_dte_options = []
    for option_symbol, option_data in options_chain.items():
        if today_str in option_symbol:
            zero_dte_options.append(option_data)
    return zero_dte_options

# Function to place a trade based on criteria
def trade_0DTE_options(symbol):
    zero_dte_options = get_0DTE_options(symbol)
    if zero_dte_options:
        for option in zero_dte_options:
            # Example criteria: Place a call trade if the option is at-the-money
            if option.strike == 
            if option.strike == trade_client.get_latest_trade(symbol).price:
                print(f"Placing order for {option.symbol}")
                # Place a market buy order
                api.submit_order(
                    symbol=option.symbol,
                    qty=1,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )
                # Risk management: Implement stop-loss or take-profit conditions

# Main loop to monitor and trade 0DTE options
if __name__ == "__main__":
    symbol = 'AAPL'  # Replace with your stock symbol
    while True:
        try:
            trade_0DTE_options(symbol)
            time.sleep(60)  # Check every minute
        except Exception as e:
            print(f"Error: {e}")


In [ ]:
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 10,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

In [1]:
from alpaca.data.historical import StockHistoricalDataClient, OptionHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, OptionBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

In [2]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

from dotenv import load_dotenv
import os

# keys is stored in config.env file
# - Specify the path to the config.env file
dotenv_path = os.path.join(os.getcwd(), "config.env")
# Load environment variables from the specified .env file
load_dotenv(dotenv_path)

api_key = os.getenv("ALPACA_API_KEY_PAPER")
secret_key = os.getenv('ALPACA_API_SECRET_KEY_PAPER')
base_url = os.getenv('ALPACA_BASE_URL')

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None